In [3]:
import zipfile

zf = zipfile.ZipFile('final_dataset_2.zip')
zf.extractall('http://34.66.5.18:8888/files/')
zf.close()

In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from PIL import Image

In [7]:
# setting the dimensions of the images
img_width, img_height = 150, 150

In [8]:
# data loading
train_data_dir = 'http://34.66.5.18:8888/files/final_dataset_2/train'
validation_data_dir = 'http://34.66.5.18:8888/files/final_dataset_2/test'
batch_size = 64

In [9]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
# data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
# data preparation
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

Found 29760 images belonging to 2 classes.
Found 7439 images belonging to 2 classes.


In [14]:
# training
nb_epochs = 20

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator, 
    validation_steps=validation_generator.samples // batch_size,
    epochs=nb_epochs)

Epoch 1/20
465/465 [==============================] - 217s 466ms/step - loss: 0.4617 - acc: 0.7844 - val_loss: 0.5004 - val_acc: 0.7311
Epoch 2/20
465/465 [==============================] - 210s 452ms/step - loss: 0.3712 - acc: 0.8415 - val_loss: 0.6034 - val_acc: 0.6834
Epoch 3/20
465/465 [==============================] - 212s 456ms/step - loss: 0.3108 - acc: 0.8777 - val_loss: 0.8289 - val_acc: 0.6182
Epoch 4/20
465/465 [==============================] - 210s 451ms/step - loss: 0.2679 - acc: 0.8996 - val_loss: 0.4968 - val_acc: 0.8032
Epoch 5/20
465/465 [==============================] - 209s 449ms/step - loss: 0.2331 - acc: 0.9143 - val_loss: 0.7307 - val_acc: 0.7065
Epoch 6/20
465/465 [==============================] - 211s 453ms/step - loss: 0.2070 - acc: 0.9263 - val_loss: 0.8482 - val_acc: 0.6924
Epoch 7/20
465/465 [==============================] - 210s 452ms/step - loss: 0.1847 - acc: 0.9362 - val_loss: 0.5117 - val_acc: 0.7856
Epoch 8/20
465/465 [============================

In [15]:
model.save('second_try.h5')